##Spotify Most Streamed Songs EDA & Prediction

**Main goal of a project** is to make an EDA of dataset and to find which regression model will represent better on this data.

**Technologies & library used:** pandas, numpy, seaborn, plotly, matplotlib for analysis and plotting; Random Forest, Gradient Boosting Machines (GBM), Support Vector Machines (SVM), Logistic Regression and TensorFlow Keras Neural Networks for models.

**Main info about dataset:**

Dataset is from [Kaggle](https://www.kaggle.com/datasets/amaanansari09/most-streamed-songs-all-time) and it shows 100 most streamed songs on Spotify with their Features Extracted using the Spotify API. 

Firstly it consists of 2 datasets 'features' and 'streams'. 

Dataset 'Features' has columns such as:
* *id* - Unique ID given to each song on spotify;
* *name* - name of the song;
* *duration* - duration of the song in minutes
* *energy* - measure from 0.0 to 1.0 and represents a perceptual measure of intensity and activity. Perceptual features contributing to this attribute include dynamic range, perceived loudness, timbre, onset rate, and general entropy.
* *key* - the key the track is in. Integers map to pitches using standard Pitch Class notation. E.g. 0 = C, 1 = C♯/D♭, 2 = D, and so on. If no key was detected, the value is -1.
* *loudness* - the overall loudness of a track in decibels (dB). Loudness values are averaged across the entire track and are useful for comparing relative loudness of tracks. Loudness is the quality of a sound that is the primary psychological correlate of physical strength (amplitude). Values typically range between -60 and 0 db.
* *mode* - indicates the modality (major or minor) of a track, the type of scale from which its melodic content is derived. Major is represented by 1 and minor is 0.
* *speechiness* - detects the presence of spoken words in a track. The more exclusively speech-like the recording (e.g. talk show, audio book, poetry), the closer to 1.0 the attribute value. Values above 0.66 describe tracks that are probably made entirely of spoken words. Values between 0.33 and 0.66 describe tracks that may contain both music and speech, either in sections or layered, including such cases as rap music. Values below 0.33 most likely represent music and other non-speech-like tracks.
* *acousticness* - a confidence measure from 0.0 to 1.0 of whether the track is acoustic. 1.0 represents high confidence the track is acoustic.
* *instrumentalness* - predicts whether a track contains no vocals. "Ooh" and "aah" sounds are treated as instrumental in this context. Rap or spoken word tracks are clearly "vocal". The closer the instrumentalness value is to 1.0, the greater likelihood the track contains no vocal content. Values above 0.5 are intended to represent instrumental tracks, but confidence is higher as the value approaches 1.0.
* *liveness* - represents the probability that a song was performed live. A value close to 1 indicates a high likelihood of the song being performed live, while a value close to 0 suggests a studio recording.
* *valence*	- represents the musical positiveness conveyed by a song. A higher valence value indicates a more positive and cheerful tone, while a lower value represents a more negative or sad tone.
* *tempo*	- represents the tempo or speed of a song measured in beats per minute (BPM). It indicates the overall pace and rhythm of the music.
* *danceability* - measures the suitability of a song for dancing based on factors like rhythm, beat strength, and tempo. A higher danceability value suggests a song that is more suitable for dancing, while a lower value indicates a song that may be less danceable.

Dataset 'Streams' has such columns:
* *song* - name of the song
* *artist* - name of the artist
* *streams(billions)* - streams of the song in billions 
* release date - date of the song release



####Importing libraries

In [ ]:
import pandas as pd
import numpy as np 
import seaborn as sb
import matplotlib.pyplot as plt
import plotly.express as px
from datetime import datetime

####Importing dataset and merging it

In [5]:
features = pd.read_csv('/kaggle/input/most-streamed-songs-all-time/Features.csv')
streams = pd.read_csv('/kaggle/input/most-streamed-songs-all-time/Streams.csv')

In [ ]:
features.head()

In [ ]:
streams.head()

In [ ]:
df_main = pd.merge(left=streams, left_on = 'Song',
                   right = features, right_on = 'name')

In [ ]:
df_main.head(10)

####Basic data analysis

Dropping column 'name' which is duplicate of the column 'song'

In [ ]:
df_main = df_main.drop(columns = ['name'])

Checking missing values

In [ ]:
print(f'Amount of missing values:\n{df_main.isna().sum()}')

Checking duplicates

In [ ]:
print(f'Amount of duplicates: {df_main.duplicated().sum()}')

Main info about our final data

In [ ]:
df_main.info()

In [ ]:
df_main.describe()

Creating a column with counting days from release date

In [ ]:
df_main['Release Date'] = pd.to_datetime(df_main['Release Date'])


today = datetime.today().strftime('%Y-%m-%d')
df_main['Days Since Release'] = (pd.to_datetime(today) - df_main['Release Date']).dt.days

In [ ]:
df_main.head()

####EDA

Scatterplot of release date and days since release

In [ ]:

fig = px.scatter(df_main, x="Days Since Release", y="Song", color = 'Days Since Release')
fig.show()

Correlation Matrix

In [ ]:
numerical_cols = ['duration', 'energy', 'loudness', 'speechiness',
                  'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'danceability', 'key', 'mode']

In [ ]:
corr_matrix = df_main[numerical_cols].corr()

plt.imshow(corr_matrix, cmap='coolwarm', interpolation='none')
plt.colorbar()
plt.xticks(range(len(corr_matrix)), corr_matrix.columns, rotation=90)
plt.yticks(range(len(corr_matrix)), corr_matrix.columns)
plt.title('Correlation Matrix')
plt.show()

Distribution of duration, energy, loudness, speechiness, acousticness, instrumentalness, liveness, valence, tempo and danceability by artist

In [ ]:
cols_to_compare = df_main[['duration', 'energy', 'loudness','speechiness', 
                           'acousticness', 'instrumentalness', 'liveness', 
                           'valence', 'tempo', 'danceability']]

In [ ]:
for feature in cols_to_compare:
    fig = px.violin(df_main, x='Artist', y=feature,  color = 'Artist', box=True, points='all')
    fig.update_layout(title=f"Distribution of {feature} by Artist",
                      xaxis_title="Artist",
                      yaxis_title=feature)
    fig.show()

Scatter plot of Danceability and Valence

In [ ]:
colors = np.array(df_main['Streams (Billions)'])
plt.scatter(df_main['danceability'], df_main['valence'], c=colors, cmap='viridis')
plt.title("Danceability & Valence")
plt.xlabel("Danceability")
plt.ylabel("Valence")
plt.show()

Scatter plot of Loudness and Energy

In [ ]:
colors = np.array(df_main['Streams (Billions)'])
plt.scatter(df_main['loudness'], df_main['energy'], c=colors, cmap='viridis')
plt.title("Loudness & Energy")
plt.xlabel("Loudness")
plt.ylabel("Energy")
plt.show()

Top 10 Most Streamed Artists

In [ ]:
# grouping by artist and summming streams and then sort 
df_grouped = df_main.groupby('Artist').agg({'Streams (Billions)': 'sum'})

df_sorted = df_grouped.sort_values('Streams (Billions)', ascending=False)


top_artists = df_sorted.head(10)

colors = ['tab:blue', 'tab:orange', 'tab:green', 'tab:red', 'tab:purple', 
          'tab:brown', 'tab:pink', 'tab:gray', 'tab:olive', 'tab:cyan']

# creating bar plot
plt.barh(top_artists.index, top_artists['Streams (Billions)'], color = colors)
plt.title('Top 10 Most Streamed Artists')
plt.xlabel('Total Streams')
plt.ylabel('Artist')
plt.show()

####Building Regression Models

**Models which will be used:**
* Random Forest
* Gradient Boosting Machines (GBM)
* Support Vector Machines (SVM)
* Logistic Regression
* Neural Networks (NN)

Importing libraries

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, RandomizedSearchCV
import math
from math import sqrt
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import tensorflow as tf
from tensorflow import keras
from keras.layers import Dense, Dropout
from keras.models import Sequential, load_model
from tensorflow.keras.optimizers import Adam

In [ ]:
df_main['Streams (Billions)'].value_counts()

Regression problem is to predict the streams of the song in billions using all the features in dataset without name of the artist, song and its release date. The target variable will be the column of streams of the song in billions.

In [ ]:
x = df_main.drop(["Artist", "Streams (Billions)", "Release Date", 'Days Since Release', 'id', 'Song'], axis=1)
y = df_main["Streams (Billions)"]

In [ ]:
scaler = StandardScaler()
x_scaled = scaler.fit_transform(x)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x_scaled, y, test_size=0.2, random_state=42)

#####**Random Forest**

In [ ]:
rf_reg = RandomForestRegressor(random_state = 42)

rf_reg.fit(X_train, y_train)

y_pred = rf_reg.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

mae = mean_absolute_error(y_test, y_pred)
print("Mean Absolute Error:", mae)

rmse = mean_squared_error(y_test, y_pred, squared=False)
print("Root mean squared error:", rmse)

r2 = r2_score(y_test, y_pred)
print("R-squared:", r2)

Tuning Random Forest Regressor using Randomized Search

In [ ]:
n_estimators = [int(x) for x in np.linspace(start = 200, 
                                            stop = 2000, 
                                            num = 10)]
max_features = ['auto', 'sqrt']
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4]
bootstrap = [True, False]

random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

rf_reg_tuned = RandomizedSearchCV(estimator = rf_reg, 
                                  param_distributions = random_grid,
                                  n_iter = 100,
                                  cv = 3,
                                  verbose = 2,
                                  random_state = 42,
                                  n_jobs = -1)

rf_reg_tuned.fit(X_train, y_train)

In [ ]:
rf_reg_tuned.best_params_

Creating a function to evaluate models using metrics:
* Mean Squared error
* Mean Absolute Error
* Root Mean squared error
* R-squared

In [ ]:
def evaluate(model, test_features, test_labels):
  prediction = model.predict(test_features)
  mse = mean_squared_error(test_labels, prediction)
  mae = mean_absolute_error(test_labels, prediction)
  rmse = mean_squared_error(test_labels, prediction, squared=False)
  r2 = r2_score(test_labels, prediction)
  print("Mean Squared Error:", mse)
  print("Mean Absolute Error:", mae)
  print("Root mean squared error:", rmse)
  print("R-squared:", r2)
  return mse

In [ ]:
best_random = rf_reg_tuned.best_estimator_

In [ ]:
results_base = evaluate(rf_reg, X_test, y_test)

In [ ]:
results_tuned = evaluate(best_random, X_test, y_test)

In [ ]:
print('Improvement of {:0.2f}%.'.format( 100 * (results_base - results_tuned) / results_base))

#####**GradientBoostingMachines**

In [ ]:
gbm_reg = GradientBoostingRegressor(random_state = 42)

gbm_reg.fit(X_train, y_train)

y_pred = gbm_reg.predict(X_test)

Tuning Gradient Boosting Regressor

In [ ]:
n_estimators = [int(x) for x in np.linspace(start = 200, 
                                            stop = 2000, 
                                            num = 10)]
max_features = ['auto', 'sqrt']
learning_rate = [0.01, 0.2]
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4]


random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'learning_rate': learning_rate,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               }

gbm_reg_tuned = RandomizedSearchCV(estimator = gbm_reg, 
                                  param_distributions = random_grid,
                                  n_iter = 100,
                                  cv = 3,
                                  verbose = 2,
                                  random_state = 42,
                                  n_jobs = -1)

gbm_reg_tuned.fit(X_train, y_train)

In [ ]:
best_random = gbm_reg_tuned.best_estimator_

In [ ]:
results_base = evaluate(gbm_reg, X_test, y_test)

In [ ]:
results_tuned = evaluate(best_random, X_test, y_test)

In [ ]:
print('Improvement of {:0.2f}%.'.format( 100 * (results_base - results_tuned) / results_base))

#####**Support Vector Machines (SVM)**

In [ ]:
svm_reg = SVR()

svm_reg.fit(X_train, y_train)

y_pred = svm_reg.predict(X_test)

Tuning Support Vector Machine

In [ ]:

param_grid = {
    'C': np.logspace(-3, 3, 7),
    'epsilon': [0.1, 0.2, 0.5, 1.0],
    'kernel': ['linear', 'rbf', 'poly'],
    'gamma': ['scale', 'auto']
    }

svm_reg_tuned = RandomizedSearchCV(estimator=svm_reg, 
                                   param_distributions=param_grid,
                                   n_iter=100, 
                                   cv=3, 
                                   verbose=2, 
                                   random_state=42, 
                                   n_jobs=-1)


svm_reg_tuned.fit(X_train, y_train)

In [ ]:
best_params = svm_reg_tuned.best_params_
print("Best Hyperparameters:", best_params)

y_pred = svm_reg_tuned.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

In [ ]:
best_random = svm_reg_tuned.best_estimator_
print(best_random)

In [ ]:
results_base = evaluate(svm_reg, X_test, y_test)

In [ ]:
results_tuned = evaluate(best_random, X_test, y_test)

In [ ]:
print('Improvement of {:0.2f}%.'.format( 100 * (results_base - results_tuned) / results_base))

#####**Logistic Regression**

In [ ]:
lg_reg = LogisticRegression()

lg_reg.fit(X_train, y_train.astype('int'))

Tuning Logistic Regression model


In [ ]:
param_grid = {
    'C': np.logspace(-4, 4, 20),  
    'penalty': ['l1', 'l2'],       
    'solver': ['liblinear']        
}

lg_reg_tuned = RandomizedSearchCV(estimator=lg_reg, 
                                   param_distributions=param_grid, 
                                   n_iter=100,  
                                   cv=5,        
                                   random_state=42,
                                   n_jobs=-1)

lg_reg_tuned.fit(X_train, y_train.astype('int'))

In [ ]:
best_params = lg_reg_tuned.best_params_
best_random = lg_reg_tuned.best_estimator_
print(best_params)
print(best_random)

In [ ]:
lg_reg_tuned = LogisticRegression(penalty = 'l1',
                                  C = 0.23357214690901212,
                                  solver = 'liblinear')

In [ ]:
lg_reg_tuned.fit(X_train, y_train.astype('int'))

results_tuned = evaluate(lg_reg_tuned, X_test, y_test)

In [ ]:
results_base = evaluate(lg_reg, X_test, y_test)

In [ ]:
print('Improvement of {:0.2f}%.'.format( 100 * (results_base - results_tuned) / results_base))

#####Neural Network

Creating base network

In [ ]:
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dense(32, activation='relu'))
model.add(Dense(1))  

model.compile(loss='mean_squared_error', optimizer='adam')

model.fit(X_train, y_train, epochs=50, batch_size=32, verbose=1)

mse = model.evaluate(X_test, y_test)
print("Mean Squared Error:", mse)

y_pred = model.predict(X_test)

Tuning Neural Network

In [ ]:
model_tuned = Sequential()
model_tuned.add(Dense(64, activation='relu', input_shape=(X_train.shape[1],)))  
model_tuned.add(Dropout(0.2))  # Dropout layer to prevent overfitting
model_tuned.add(Dense(64, activation='relu'))  # Hidden layer
model_tuned.add(Dropout(0.2))  
model_tuned.add(Dense(1, activation='linear'))  

model_tuned.compile(loss='mean_squared_error', optimizer=Adam(lr=0.001))

history = model_tuned.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test))

mse_tuned = model_tuned.evaluate(X_test, y_test)
print("Mean Squared Error:", mse)

In [ ]:
print(mse)

In [ ]:
print(mse_tuned)

In [ ]:
print('Improvement of {:0.2f}%.'.format(100 * (mse - mse_tuned) / mse))

####What model is the best?

In [ ]:
mse_values = [0.20920275288513707, 0.21675758635354847, 0.25346728762066706, 1.07842985, 0.6379622220993042]  


model_names = ['RF', 'GBM', 'SVM', 'LR', 'NN']  


plt.bar(model_names, mse_values)
plt.xlabel('Models')
plt.ylabel('MSE')
plt.title('Comparison of MSE Values')
plt.show()

####Conclusion

As we can see, the best model to use with a regression problem (predicting Streams in billions of top songs) on this dataset is **Random Forest** with the lowest MSE, the worst is **Logistic Regression**.

😀


